In [2]:
# Traceable Neural Network with Fixed Target Backpropagation
import math

class Neuron:
    def __init__(self, bias=None):
        self.bias = bias
        self.weights = []

class NeuronLayer:
    def __init__(self, num_neurons, bias=None):
        self.neurons = [Neuron(bias) for _ in range(num_neurons)]

class NeuralNetwork:
    def __init__(self):
        self._configure_network()

    def _configure_network(self):
        print("=== CONFIGURE YOUR NEURAL NETWORK ===")
        self.num_inputs = int(input("Enter number of input neurons: "))
        self.num_hidden_layers = int(input("Enter number of hidden layers: "))
        self.neurons_per_hidden = []
        self.hidden_layers = []

        for i in range(self.num_hidden_layers):
            n = int(input(f"Enter number of neurons in hidden layer {i + 1}: "))
            self.neurons_per_hidden.append(n)
            self.hidden_layers.append(NeuronLayer(n))

        self.num_outputs = int(input("Enter number of output neurons: "))
        self.output_layer = NeuronLayer(self.num_outputs)

        self._initialize_weights()

    def _initialize_weights(self):
        print("\n=== SET WEIGHTS FOR EACH CONNECTION ===")

        print(f"\nInput Layer → Hidden Layer 1")
        for i, neuron in enumerate(self.hidden_layers[0].neurons):
            neuron.weights = [
                float(input(f"Weight from Input {j + 1} to Hidden1 Neuron {i + 1}: "))
                for j in range(self.num_inputs)
            ]

        for l in range(1, self.num_hidden_layers):
            print(f"\nHidden Layer {l} → Hidden Layer {l + 1}")
            prev_layer = self.hidden_layers[l - 1]
            for i, neuron in enumerate(self.hidden_layers[l].neurons):
                neuron.weights = [
                    float(input(f"Weight from Hidden{l} Neuron {j + 1} to Hidden{l + 1} Neuron {i + 1}: "))
                    for j in range(len(prev_layer.neurons))
                ]

        print(f"\nHidden Layer {self.num_hidden_layers} → Output Layer")
        for i, neuron in enumerate(self.output_layer.neurons):
            neuron.weights = [
                float(input(f"Weight from Hidden{self.num_hidden_layers} Neuron {j + 1} to Output Neuron {i + 1}: "))
                for j in range(len(self.hidden_layers[-1].neurons))
            ]

    def sigmoid(self, z):
        return 1 / (1 + math.exp(-z))

    def sigmoid_derivative(self, a):
        return a * (1 - a)

    def full_forward_pass(self, inputs):
        print("\n=== DETAILED FORWARD PASS ===")
        activations = [inputs]
        for l, layer in enumerate(self.hidden_layers):
            prev_activations = activations[-1]
            current_outputs = []
            print(f"\n[Layer {l} → Layer {l+1}]")
            for i, neuron in enumerate(layer.neurons):
                z = sum(w * x for w, x in zip(neuron.weights, prev_activations))
                a = self.sigmoid(z)
                current_outputs.append(a)
                print(f"h{l+1}{i+1}: z = ({' + '.join(f'{w:.2f}*{x}' for w,x in zip(neuron.weights, prev_activations))}) = {z:.4f}")
                print(f"      a = σ({z:.4f}) = {a:.4f}")
            activations.append(current_outputs)

        print(f"\n[Hidden{self.num_hidden_layers} → Output]")
        prev_activations = activations[-1]
        output_activations = []
        for i, neuron in enumerate(self.output_layer.neurons):
            z = sum(w * h for w, h in zip(neuron.weights, prev_activations))
            a = self.sigmoid(z)
            output_activations.append(a)
            print(f"y{i+1}: z = ({' + '.join(f'{w:.2f}*{h:.4f}' for w, h in zip(neuron.weights, prev_activations))}) = {z:.4f}")
            print(f"     a = σ({z:.4f}) = {a:.4f}")
        activations.append(output_activations)
        return activations

    def full_backward_pass(self, activations, target):
        print("\n=== DETAILED BACKWARD PASS ===")

        output_activations = activations[-1]
        errors = [y - target for y in output_activations]
        deltas = []
        print("\n[Output Layer]")
        for i, (y, err) in enumerate(zip(output_activations, errors)):
            dσ_dz = self.sigmoid_derivative(y)
            δ = err * dσ_dz
            deltas.append(δ)
            print(f"Using fixed target value = {target}")
            print(f"Error = y - target = {y:.4f} - {target} = {err:.4f}")
            print(f"∂σ/∂z = y*(1-y) = {y:.4f}*{1-y:.4f} = {dσ_dz:.4f}")
            print(f"δ_output = Error * ∂σ/∂z = {err:.4f} * {dσ_dz:.4f} = {δ:.4f}")

        layer_deltas = [deltas]
        for l in reversed(range(self.num_hidden_layers)):
            print(f"\n[Hidden{l+1} Gradients]")
            current_layer = self.hidden_layers[l]
            next_deltas = layer_deltas[0]
            if l == self.num_hidden_layers - 1:
                next_layer = self.output_layer
            else:
                next_layer = self.hidden_layers[l + 1]

            layer_delta = []
            for i, neuron in enumerate(current_layer.neurons):
                weighted_sum = sum(
                    next_deltas[j] * next_layer.neurons[j].weights[i]
                    for j in range(len(next_deltas))
                )
                a = activations[l + 1][i]
                dσ_dz = self.sigmoid_derivative(a)
                δ = weighted_sum * dσ_dz
                layer_delta.append(δ)
                print(f"h{l+1}{i+1}:")
                print(f"  Weighted sum = {weighted_sum:.6f}")
                print(f"  ∂σ/∂z = a*(1-a) = {a:.4f}*{1-a:.4f} = {dσ_dz:.4f}")
                print(f"  δ = Sum * ∂σ/∂z = {weighted_sum:.6f} * {dσ_dz:.4f} = {δ:.6f}")
            layer_deltas.insert(0, layer_delta)

        print("\n[Input Gradients]")
        input_grads = []
        for i in range(self.num_inputs):
            grad = sum(
                layer_deltas[0][j] * self.hidden_layers[0].neurons[j].weights[i]
                for j in range(len(self.hidden_layers[0].neurons))
            )
            print(f"∂E/∂x{i+1} = {' + '.join(f'{layer_deltas[0][j]:.6f}*{self.hidden_layers[0].neurons[j].weights[i]:.2f}' for j in range(len(self.hidden_layers[0].neurons)))} = {grad:.6f}")
            input_grads.append(grad)

# Run the network
nn = NeuralNetwork()
inputs = list(map(float, input("\nEnter input values (space-separated): ").split()))
target = 1.0  # Fixed target
print("\n=== NEURAL NETWORK EXECUTION ===")
print(f"Network Architecture: {nn.num_inputs}-{'-'.join(map(str, nn.neurons_per_hidden))}-{nn.num_outputs}")
print(f"Input Values: {inputs}")
print("Using fixed target = 1.0 for backpropagation (ignoring actual output error)")
activations = nn.full_forward_pass(inputs)
nn.full_backward_pass(activations, target)
1


=== CONFIGURE YOUR NEURAL NETWORK ===
Enter number of input neurons: 1
Enter number of hidden layers: 2
Enter number of neurons in hidden layer 1: 1
Enter number of neurons in hidden layer 2: 1
Enter number of output neurons: 3

=== SET WEIGHTS FOR EACH CONNECTION ===

Input Layer → Hidden Layer 1
Weight from Input 1 to Hidden1 Neuron 1: 1

Hidden Layer 1 → Hidden Layer 2
Weight from Hidden1 Neuron 1 to Hidden2 Neuron 1: 0.5

Hidden Layer 2 → Output Layer
Weight from Hidden2 Neuron 1 to Output Neuron 1: 1
Weight from Hidden2 Neuron 1 to Output Neuron 2: 1.5
Weight from Hidden2 Neuron 1 to Output Neuron 3: 1

Enter input values (space-separated): 1

=== NEURAL NETWORK EXECUTION ===
Network Architecture: 1-1-1-3
Input Values: [1.0]
Using fixed target = 1.0 for backpropagation (ignoring actual output error)

=== DETAILED FORWARD PASS ===

[Layer 0 → Layer 1]
h11: z = (1.00*1.0) = 1.0000
      a = σ(1.0000) = 0.7311

[Layer 1 → Layer 2]
h21: z = (0.50*0.7310585786300049) = 0.3655
      a =

1